In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from code.mpmath_series import quad_phi
from code.mpmath_series import series_x_mu_b0_pos, bound_series_x_mu_b0_pos, estimate_N_series_x_mu_b0_pos
from code.mpmath_series import series_x_mu_b0_alternating, bound_series_x_mu_b0_alternating, estimate_N_series_x_mu_b0_alternating

from code.mpmath_series import estimate_N_series_x_mu_b0_pos_fast, estimate_Tk_series_x_mu_b0_pos_fast

In [3]:
from mpmath import *

In [243]:
mp.dps = 50

alpha = mpf('1/100')
mu = mpf('1/5')
delta = mpf('1/100')
x = mpf('20')

(x - mu) / delta

omega = sqrt((x-mu)**2 + delta**2)
float(alpha * omega), float(((x-mu)**2  / omega * alpha)), float(omega**2)

(0.19800002525252364, 0.19799997474747957, 392.0401)

### 1. Integral

In [244]:
ref = quad_phi(x, alpha, mu, delta, digits=100)
ref

mpf('0.99987901257027552272415620085010655116346186377510146')

### 2. Series

In [245]:
estimate_N_series_x_mu_b0_pos(x, alpha, mu, delta)

0.999999872462018 8.038930241482221e-05 4.925537242961908e-25 1.0357288255129176e+18 74010078.13672733


mpf('84914922.865642436614692979600042362158984603952325332')

In [103]:
bound_series_x_mu_b0_pos(x, alpha, mu, delta, 1494)

2.1723187395685342e-16

In [104]:
float(ref - series_x_mu_b0_pos(x, alpha, mu, delta, 1494))

2.140548912670144e-16

Series

In [105]:
omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega

N = 50

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

remainder = mp.zero
for k in range(N, 1494):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    remainder += r * q

print(s, remainder)

5.301265214351715494021704346621584674595469818635 1.0642227718258776694697124025818633803761076268033


Remainder integral representation

In [106]:
M = gamma(N + 3 / 2) * 2 / fac2(2*N + 1) * (2*alpha / omega) / sqrt(pi) * (2*z*alpha / omega)**N
M * quad(lambda t: cos(omega*t) / (t**2 + alpha**2)**(N + 3/2) / (2 - 2*z*alpha / omega / (t**2 + alpha**2)), [0, inf])

mpf('1.0617314391410902894462675697808911643647083789664338')

In [248]:
N = 14

omega = sqrt(delta**2 + (x-mu)**2)
z = (x-mu)**2 * alpha / omega
aw = alpha * omega

s = mp.zero
for k in range(N):
    r = z**k / fac2(2*k + 1)
    q = besselk(k + 1, alpha * omega)
    s += r * q

sN = s

In [249]:
m = 2*z*alpha/omega

S1 = gamma(N + 3 / 2) / fac2(2*N + 1) * (alpha / omega) * sqrt(pi) * 2**(N+1) / sqrt(2*alpha**2*m - m*m) * cosh(sqrt(omega**2 * (2*alpha**2 - m)) / sqrt(2))

S2 = 0
for k in range(N):
    P = alpha**(2*k) * gamma(N + 1 - k) * gamma(k - 1/2) * hyp2f1(1, N + 1 - k, 3/2 - k, 1 - m/(2*alpha**2))
    tk = (-1)**k * omega**(2*k) / factorial(2*k) * P
    S2 += tk
    print(k, tk)

S2 *= (m * 2)**N * alpha**(-2*N -1) / pi / fac2(2*N - 1) * gamma(N + 3 / 2) / fac2(2*N + 1) / omega

S = sN + S1 + S2

S * delta * exp(delta * alpha) / pi  * (x-mu)*alpha / omega  + 1/2

0 -309039784191.03900701510932284097160392881132757348
1 -216350971.73237263971273770898005881119430338582779
2 27184.964198907226786738890725470390596034681590877
3 -4.440685423727143403416717725350898344843600594358
4 0.0007065455991140312140082898757955915321842018532278
5 -0.00000010772001191032342860570457038107976631093735417851
6 0.000000000015996429346384906754581084837838257457085567331371
7 -0.0000000000000023689446759768654485781454433474730408217753847347
8 3.5932673102897461686426032093121948317943461190166e-19
9 -5.7545137874123767181319700285861899849009236846401e-23
10 1.0092632842891938980301964116538677779643624305429e-26
11 -2.0340265745864905282110791568050834878275893421829e-30
12 5.0561051309982294442617550729751070066491929949059e-34
13 -1.7534810925762908090497565712998678633532474391377e-37


mpf('0.99987901257027552272415620085010655116346186444698896')

In [250]:
ref

mpf('0.99987901257027552272415620085010655116346186377510146')

Find $N$

minimum value

In [92]:
(alpha*omega)**(2*N) * gamma(N - 1/2) / factorial(2*N)

mpf('4.3614613310291277381292558823510364548369601571451268e-51')

In [93]:
(alpha*omega)**(2*N) * 2**(-2*N - 1/2) * (e / N)**N

mpf('3.8193639553596949103299692830615390665248245158608532e-49')

In [246]:
epsilon = 1e-16
# (alpha * omega / 2)**(2*N) * (e / N)**N = epsilon * sqrt(2)

In [247]:
-log(epsilon * sqrt(2)) / lambertw(-log(epsilon * sqrt(2)) / e ** (alpha* omega / 2))

mpf('14.263139206139716885692917547746032635929929646695991')

In [190]:
1 - m/(2*alpha**2)

mpf('0.0000062499609377441390991306304335597902513109293085216299')